# ResNet Modeling

## Dataset: cifar 10
## Model: ResNet v2
- **ResNet v1** :  [Deep Residual Learning for Image Recognition, 2015](https://arxiv.org/pdf/1512.03385.pdf)
- **ResNet v2** : [Identity Mapping in Deep Residual Networks, 2016](https://arxiv.org/pdf/1603.05027)

## Experiments
- **ResNet v2** model로 실습, hyperparameter 변경 실습
  * epoch 
  * ResNet block 수 n  
  * subtract_pixel_mean = True/False 변경 
- inference에서 test image 사용한 결과 보기
- 끝 셀까지 실행해 주세요. 학습후 모델 저장 후 로드하여 확인까지 완료해 주세요. 
  * 저장된 모델은 다른 실습에서 활용할 예정입니다.

If necessary, install scipy
```{bash}
!pip3 install scipy
```


Reference: https://github.com/PacktPublishing/Advanced-Deep-Learning-with-Keras/blob/master/chapter2-deep-networks/resnet-cifar10-2.2.1.py

In [ ]:
pip install pydot --user

In [1]:
!nvidia-smi

Thu May 29 12:12:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 553.62                 Driver Version: 553.62         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q               WDDM  |   00000002:00:00.0 Off |                    0 |
| N/A    0C    P8             N/A /  N/A  |    4043MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# tf 2.9
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.models import Model
from keras.datasets import cifar10

import numpy as np
import os
import math
from datetime import datetime 

In [3]:
!nvidia-smi

Thu May 29 12:12:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 553.62                 Driver Version: 553.62         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A10-24Q               WDDM  |   00000002:00:00.0 Off |                    0 |
| N/A    0C    P8             N/A /  N/A  |    4031MiB /  24512MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import numpy as np
import os
os.environ["KERAS_BACKEND"] = 'torch'

import matplotlib.pyplot as plt

In [4]:
import keras

keras_major = int(keras.version().split('.')[0])

AttributeError: module 'keras' has no attribute 'version'

In [5]:
keras_major = int(keras.__version__.split('.')[0])
keras_major

2

In [ ]:
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
# migration to keras 3.0
#from keras.preprocessing.image import ImageDataGenerator
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.models import Model
from keras.datasets import cifar10

import numpy as np
import os
import math
from datetime import datetime 

In [ ]:
device = None
if keras_major == 3:
    import torch

    device = "cuda" if torch.cuda.is_available() else "cpu"
    
# device = "cpu"
#device = None
print(device)

In [6]:
# for display image and plot
from PIL import Image
import matplotlib.pyplot as plt

## 0. Configuration & Hyperparameters

In [7]:
num_classes = 10 # cifar10 classes : fixed
data_augmentation = True

# Subtracting pixel mean can improve accuracy
# Do this if True
subtract_pixel_mean = True

batch_size = 32  # 128  # original paper trained all networks with batch_size=128
epochs = 120     # 100, 120(~55m, 91%), 150, 200(1:30m), 300(2:30m, 91%)

In [8]:
# Model version
version = 2      # fixed, 본 실습에서는 ResNet-v2 모델 사용합니다.

# depth : layer 수
# v1 n : number of residual blocks                        
#     { 3,  5,  7,  9, ...}. cf. training time
# v1 depth: the corresponding number of layers(depth): 
#           {20, 32, 44, 56, ... }
# v2 n : number of residual blocks                        
#     { 2,  3,  4,  5,  6,  7,  9, ...}. cf. training time
# v2 depth: the corresponding number of layers(depth): 
#     {20, 29, 38, 47, 56, 65, 83, ... }

n = 2 # 변경 가능하지만, 증가할 수록 학습 시간이 오래 걸립니다.
# Computed depth from supplied model parameter n
# 1 input Conv, last dense layer : 2 extra layers
if version == 1:
    # 3 stages x 2 (conv layers / ResNetBlock)  x n ((number of ResNetBlocks)/stage)
    depth = n * 6 + 2 
elif version == 2:
    # 3 stage x 3 (conv layers/ResNetBlock) x n ((number of ResNetBlocks)/stage)
    depth = n * 9 + 2 
    
print(f'model: ResNet-v{version}-{depth}')

model: ResNet-v2-20


## 1. Dataset Preparation
- Load cifar10 dataset
- Normalize input(x) data
- Output encoding to one-hot vector

In [9]:
# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [10]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## 2. Modeling: ResNet
- Using functional APIs: Add 
  * H(x) = F(x) + x

### resnet_layer : a conv-layer w/ BN, activation
- ResNet v1, v2 모델링 시에 하나의 conv layer 구성을 표현하는 데 사용, 
  conv, normalization, activation 순서에 따라 2가지 type이 있음
  1. conv 먼저 수행 **후 normalization**하는 <span style="color:blue">**post-normalization, post-activation**</span> 구조(conv_first=True)
     * layer 구조: conv2d-layer + normalization(BN) + activation(relu)
     * input에 대해 수행하는 첫번째 conv layer
     * 사용예: image input 입력 받은 첫번째 conv-layer, ResNet v1 block 
  2. conv 수행 **전 normalization**하는 <span style="color:blue">**pre-normalization, pre-activation**</span> 구조(conv_first=False)
     * layer 구조: normalization(BN) + activation(relu) + conv2d-layer
     * **v2: normalization(BN) + activation(BN) + conv-layer**  
     * 사용시 주의점: 이후 마지막 layer에서 normalization+activation 추가 필요
     * 사용예: ResNet v2 block (3-(conv-)layer bottleneck building block)
  

In [11]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True: v1) or
            bn-activation-conv (False: v2)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first: # input conv or v1
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else: # v2, pre-normalization, pre-activation 
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x) 
    return x


## Modeling: ResNet v1
- input block(first conv.) : Conv+BN+ReLU
- middle blocks: 3 stages ResNet blocks (32x32 - 16x16 - 8x8)
  * stage 0(32x32): 16 channels(no. of filters), n ResNet Blocks
  * stage 1(16x16): 16x2 channels, n ResNet Blocks
  * stage 2(8x8): 16x4 channels, n ResNet Blocks
- output block(last classification layer): average pooling 2d (8)- flatten - dense(softmax, 10 classes)
- depth = 2 + 6*n: number of weighted layers
  * n: ResBlocks/stage
  * input block(conv) + last dense layer : 2
  * middle blocks: 6*n = 3 stages x 2 (conv layers/ResNet Block) * n (ResBlocks/stage)
  
### Residual block F(x) and projection shortcut H(x)
1) idenity shortcut: **H(x) = F(x) + x**
  - F(x) = f( resnet_layer, x)
    * F(x) : 2 or 3 resnet_layers  
        
2) projection shortcut: **H(x) = F(x) + Conv2d(x, stride=2)**
  - subsampling: Conv2d(stride=2)
  - resnet_layer = Conv2d - BN - Activation(relu) 
  - resolution을 1/2로 줄일 때, stride=2(filter수는 두배씩 증가시킴) 
  - conv2d(3x3), filter size = 64 (at 32 x 32)


In [12]:
def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y]) # x = x + y
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='glorot_uniform')(y) 

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

### Modeling: ResNet v2
- **Pre-activation** 방식 ResNet Block 사용
- input block: Conv+BN+ReLU (post-normalization:resnet_layer(conv_first=True))
 * Conv 한번 수행
- middle blocks: 3 stages로 구성 (32x32 (ch) - 16x16 - 8x8)
 * 각 stage별 n ResNet Blocks, 각 ResNet Block은 3 resnet_layer, 각 resnet_layer는 pre-normalization(resnet_layer(conv_first=False))
 * stage 0(HxW=32x32): 32x32 $\rightarrow$ 32x32, 16 channels(no. of input filters) $\rightarrow$ 64 channels(no for output filters), n ResNet Blocks
   - 첫번째 conv layer의 차이 주의  
   - 첫번째에는 conv(no normalization, no activation): 이어진 resnet_layer에서 normalization, activation 수행
   - 첫번째 ResNet Block은 channel 수 증가
 * stage 1(16x16): 32x32 $\rightarrow$ 16x16, 64 channels $\rightarrow$ 128 channls,  n ResNet Blocks
   - 첫번째 ResNet Block은 해상도 1/2, Conv2d(stride=2)
   - 나머지 ResNet Block은 동일 해상도, Conv2d(stride=1)
 * stage 2(8x8): 16x16 $\rightarrow$ 8x8, 128 channels $\rightarrow$ 256 channels, n ResNet Blocks
   - 첫번째 ResNet Block은 해상도 1/2, Conv2d(stride=2)
   - 나머지 ResNet Block은 동일 해상도, Conv2d(stride=1)
- output block (last classification layer) 
 * **Normalization-Activation** : pre-normalization, pre-activation 사용하였으므로, 추가 사용
 * (Global) Average Pooling 2d (pool_size=8,256d)- Flatten(**'feature'**) - Dense(softmax, 10 classes)
   - Flatten layer의 이름을 'feature' layer로 이름을 주고, feature level 분석에 활용
- layer 수(depth) 계산: 
  depth = 2 + 9*n (n: ResBlocks/stage)
 * number of weighted layers in the middle blocks = 3 convs/resnet block * n resnet block/stage * 3 stages 
 * input block(conv) + last dense layer : 2
 
### Residual block F(x) and Projection shortcut H(x)
1) idenity mapping : H(x) = **F(x)** + x
 - F(x) = f( resnet_layer, x) : ResNet Block
 - resnet block : 3 convs, **pre-normalization, pre-activation**
   * resnet_layer (Conv_First=False): pre-normalization, pre-activation
   *  (**BN** - **Activation** - Conv2d) - (BN - Activation - Conv2d) - (BN - Activation - Conv2d)
   * Activation: ReLU
 
2) projection shortcut: H(x) = F(x) + **Conv2d(x, stride=2)**
  - resolution 또는 channel 변경시 사용 (add위해 dimension 맞춰줌)
  - subsampling: Conv2d(stride=2)
  - resolution을  1/2로 줄일 때 stride=2(filter수 는 두배로 증가)
  - no normalization, no activation
  - conv2d(3x3) 


In [13]:
def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or 
    also known as bottleneck layer.
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, 
    the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, 
    while the number of filter maps is
    doubled. Within each stage, the layers have 
    the same number filters and the same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    Arguments:
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    Returns:
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 110 in [b])')
    # start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # first conv2d 
    # on input before splitting into 2 paths
    # - Conv2D + BN + ReLU
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                # 첫번째 stage에서 filter수 크게
                num_filters_out = num_filters_in * 4
                # when first layer and first stage,
                # conv layer 추가, 
                # 뒤에 이어진 resnet_layer에서 pre-normalization 사용에서 normalization, activation 수행
                if res_block == 0:  
                    activation = None
                    batch_normalization = False
            else:
                # 2, 3번째 stage에서 filter수 
                num_filters_out = num_filters_in * 2
                # when first layer but not first stage,
                # downsampling with stride=2
                if res_block == 0:
                    # downsample
                    strides = 2 

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1, # 1x1 kernel
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False) # 3x3 kernel
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1, # 1x1 kernel
                             conv_first=False)
            if res_block == 0:
                # when linear projection residual shortcut connection,
                # - match changed dims(filters, kernel_size=1, stride)
                #   * stride = 2 or 1(only if to change no. of channels )
                # - conv_first=True, activation=None, BN=False
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y]) # x = x + y

        num_filters_in = num_filters_out

    # add classifier on top.
    # v2 has BN-ReLU before Pooling
    # 채워 보세요
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten(name='feature')(x) # feature vector layer 이름 남김
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='glorot_uniform')(y) 

    # instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

## 3. Training
### 3.1 **Optimizer**: **Adam** + lr scheduler
#### Learning scheduler 사용

In [14]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    
    return lr

- **Optimizer**: **Adam** + lr scheduler

In [15]:
if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else: 
    model = resnet_v1(input_shape=input_shape, depth=depth)
    
if keras_major == 3 and device is not None:
    model = model.to(device)
    print(f'to {device}')
    
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

lr_scheduler = LearningRateScheduler(lr_schedule)

callbacks = [lr_scheduler]

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 32, 32, 16)   0           ['batch_normalization[0][0]']

In [16]:
model_arch_png = 'model_cifar10_%s.png' % model_type
keras.utils.plot_model(model, to_file=model_arch_png, show_shapes=True )

AttributeError: module 'pydot' has no attribute 'InvocationException'

### 3.2 Train the designed model
- Data Augmentation
- Traing time: ~1:30min/200 epoch, 91%(test dataset)

In [17]:
# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    start_time = datetime.now()    
    history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=False, #  True if necessary 
              callbacks=callbacks)
    end_time = datetime.now()

else: # Data Augmentation 사용
    print('Using real-time data augmentation.')
    # this will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False)

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)
    
    # epoch당 step 수 지정해서 입력하는 방법, 안쓰면 아래의 식으로 자동 계산
    steps_per_epoch = math.ceil(len(x_train) / batch_size)
    print('steps_per_epoch=',steps_per_epoch, ', len(x_train)/batch_size)=', len(x_train) / batch_size) 
    # fit the model on the batches generated by datagen.flow().
    start_time = datetime.now()    
    history = model.fit(x=datagen.flow(x_train, y_train, batch_size=batch_size),
              verbose=1, # progress bar
              epochs=epochs,
              validation_data=(x_test, y_test),
              steps_per_epoch=steps_per_epoch,
              callbacks=callbacks)
    end_time = datetime.now()

Using real-time data augmentation.
steps_per_epoch= 1563 , len(x_train)/batch_size)= 1562.5
Epoch 1/120
1563/1563 [==============================] - 319s 202ms/step - loss: 1.7604 - accuracy: 0.4872 - val_loss: 1.5812 - val_accuracy: 0.5536 - lr: 0.0010
Epoch 2/120
1563/1563 [==============================] - 323s 207ms/step - loss: 1.3496 - accuracy: 0.6182 - val_loss: 1.6293 - val_accuracy: 0.5430 - lr: 0.0010
Epoch 3/120
1563/1563 [==============================] - 341s 218ms/step - loss: 1.1734 - accuracy: 0.6773 - val_loss: 1.6548 - val_accuracy: 0.5496 - lr: 0.0010
Epoch 4/120
1563/1563 [==============================] - 326s 209ms/step - loss: 1.0614 - accuracy: 0.7145 - val_loss: 1.1006 - val_accuracy: 0.7083 - lr: 0.0010
Epoch 5/120
1563/1563 [==============================] - 346s 221ms/step - loss: 0.9886 - accuracy: 0.7400 - val_loss: 1.0327 - val_accuracy: 0.7289 - lr: 0.0010
Epoch 6/120
1563/1563 [==============================] - 341s 218ms/step - loss: 0.9353 - accuracy

KeyboardInterrupt: 

In [ ]:
elapsed_time = end_time - start_time
print(f'Training time:{elapsed_time}') # ~ 55 min

## Plot the train history

In [ ]:
# plot history.
def plot_history(h):
    epochs_ = len(h.history['loss'])
    ep = np.arange(epochs_)
    fig = plt.figure(figsize=(17,8))
    ax1 = fig.add_subplot(1,2,1)
    ax2 = fig.add_subplot(1,2,2)

    ax1.plot(ep, h.history['loss'], label='loss')
    ax1.plot(ep, h.history['val_loss'], label='val_loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss ')
    ax1.legend(bbox_to_anchor=(1,1))
    ax1.grid(True)

    ax2.plot(ep, h.history['accuracy'], label='accuracy')
    ax2.plot(ep, h.history['val_accuracy'], label='val_accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend(bbox_to_anchor=(1,1))
    ax2.grid(True)

In [ ]:
plot_history(history)

## 4. Evaluation 
### Quantitative Evaluation
- Accuracy on test dataset

In [ ]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

### Generate Confusion Matrix

In [ ]:
# Generate predictions for the test dataset.
predictions = model.predict(x_test)

# For each sample image in the test dataset, select the class label with the highest probability.
predicted_labels = [np.argmax(i) for i in predictions]

In [ ]:
y_test_integer_labels[:10], predicted_labels[:10]

In [ ]:
# Convert one-hot encoded labels to integers.
y_test_integer_labels = tf.argmax(y_test, axis=1)

# Generate a confusion matrix for the test dataset.
cm = tf.math.confusion_matrix(labels=y_test_integer_labels, predictions=predicted_labels)

# Plot the confusion matrix as a heatmap.
plt.figure(figsize=[12, 6])
import seaborn as sn
sn.heatmap(cm, annot=True, fmt='d', annot_kws={"size": 12})
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

## Inference test

In [ ]:
cifar10_categories=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
## Run inference on new data
# save an image from cifar10 dataset and load it
image_id = 0 # cifar10 test image id in number
image_file = f'./cifar10_test{image_id}.png'
keras.utils.save_img(image_file, x_test[image_id])
img = keras.utils.load_img(image_file, target_size=(32,32))

# image display
plt.imshow(img)
img_array = keras.utils.img_to_array(img)

# add a dimension to make an array ( a form of a list of images) for model.predict input
# [image1]
img_array = np.expand_dims(img_array, axis=0)

# preprocessing the input image  
img_array = img_array.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    img_array -= x_train_mean  # x_train_mean = np.mean(x_train, axis=0)

predictions = model.predict(img_array)
#score = predictions[0]

prediction = np.argmax(predictions[0]) # predicted class id : id for max scores

# categories[prediction]
print(f"ground truth_label = {cifar10_categories[np.argmax(y_test[image_id])]}, predicted_label = {cifar10_categories[prediction]}")
print(f"Similarity = {np.dot(predictions[0],y_test[image_id])}")

## Qualitative Evaluation

In [ ]:
def evaluate_model(x_dataset, y_label, model):

    class_names = ['airplane',
                   'automobile',
                   'bird',
                   'cat',
                   'deer',
                   'dog',
                   'frog',
                   'horse',
                   'ship',
                   'truck' ]
    num_rows = 3
    num_cols = 6
    
    # Retrieve a number of images from the dataset.
    data_batch = x_dataset[0:num_rows*num_cols]

    # Get predictions from model.  
    predictions = model.predict(data_batch)

    plt.figure(figsize=(20, 8))
    num_matches = 0
        
    if subtract_pixel_mean:
        data_batch += x_train_mean  # add to range [0,1] for display , x_train_mean = np.mean(x_train, axis=0)

    for idx in range(num_rows*num_cols):
        ax = plt.subplot(num_rows, num_cols, idx + 1)
        plt.axis("off")
        plt.imshow(data_batch[idx])

        pred_idx = tf.argmax(predictions[idx]).numpy()
        truth_idx = np.nonzero(y_label[idx])
            
        title = str(class_names[truth_idx[0][0]]) + " : " + str(class_names[pred_idx])
        title_obj = plt.title(title, fontdict={'fontsize':13})
            
        if pred_idx == truth_idx:
            num_matches += 1
            plt.setp(title_obj, color='g')
        else:
            plt.setp(title_obj, color='r')
                
        acc = num_matches/(idx+1)
    print("Prediction accuracy: ", int(100*acc)/100)
    
    return


In [ ]:
evaluate_model(x_test, y_test, model)

In [ ]:
# https://github.com/tensorflow/tensorflow/issues/59694
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR) # to ignore tracing functions warning messages


In [ ]:
# save the model to load the model later
model_file = 'cifar10_%s' % model_type

In [ ]:
model.save(model_file) # tf format
#model.save('%s.h5' % model_file) # h5 format

In [ ]:
# check if the saved model can be loaded 
reloaded_model = keras.models.load_model(model_file)
reloaded_model.summary()

In [ ]:
!dir cifar10_ResNet20v2

In [ ]:
model = reloaded_model